In [83]:

import os;
import json;
import builtins
import duckdb
import pandas as pd;
import matplotlib.pyplot as plt
import numpy as np

def get_dataset_from_testcase(run):
    return run[6:-2]

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge/pgm_sampled_debug'
runs = []
index_stats = []
for test_case in os.listdir(dir):
    if test_case == 'build':
        continue
    for run_name in os.listdir(os.path.join(dir, test_case, 'outputs', 'results', 'run')):
        json_file = builtins.open(os.path.join(dir, test_case, 'outputs', 'results', 'run', run_name))
        run = (json.load(json_file))
        run['dataset'] = get_dataset_from_testcase(test_case)
        runs.append(run)
df = pd.json_normalize(runs)
print(df.columns)
print(df[['command', 'spec.index.type']])

Index(['command', 'dataset', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_disk_fetch_size', 'result.inner_total_bytes_fetched',
       'result.outer_disk_fetch', 'result.outer_disk_fetch_size',
       'result.outer_total_bytes_fetched', 'spec.algo', 'spec.algo_name',
       'spec.check_checksum', 'spec.common_key', 'spec.index.search',
       'spec.index.type', 'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk'],
      dtype='object')
                                              command spec.index.type
0   numactl -N 1 -m 1 sponge/pgm_sampled_debug/bui...     flatpgm2048
1   numactl -N 1 -m 1 sponge/pgm_sampled_debug/bui...      sampledpgm
2   numactl -N 1 -m 1 sponge/pgm_sampled_debug/bui...      sampledpgm
3   numactl -N 1 -m 1 sponge/pgm_sa

In [84]:
epsilon = {
    "sampledpgm": 16 * 128 * 2,
    "pgm256": 256,
    "pgm1024": 1024,
    "pgm2048": 2048,
    "flatpgm256": 256,
    "flatpgm1024": 1024,
    "flatpgm2048": 2048,
    "flatpgm4096": 4096,
    "flatpgm8192": 8192,
    "hashjoin": 0,
    "btree": 0,
    "sj": 0,
    "sj2": 0,
    "standard_merge": 0
};
def indexType(x):
    if "sampled" in x:
        return "SAMPLED_PGM"
    if "flatpgm" in x:
        return "FLAT_PGM"
    elif "pgm" in x:
        return "PGM"
    elif "btree" in x:
        return "BTREE"
    return "NA"


df["threads"] = df["spec.num_threads"]
df["duration_sec"] = df["result.duration_ns"] / (1000000000)
df["ratio"] = df["spec.common_key"]
df["algo"] = df["spec.algo_name"]
df["join_algo"] = df["spec.algo"]
df["inner_bytes_fetched"] = df["result.inner_total_bytes_fetched"]
df["index_type"] = df["spec.algo_name"].map(lambda x: indexType(x))
df['epsilon'] = df['spec.index.type'].map(lambda x: epsilon[x])
df['checksum'] = df['result.checksum']


In [89]:
def plot_for_dataset(dataset):
    rows = duckdb.sql(
        " SELECT ratio, index_type, string_agg(checksum, ',') as c, epsilon, MEDIAN(duration_sec) as v FROM df"
        f" WHERE dataset='{dataset}'"
        " GROUP BY ratio, index_type, epsilon"
        " ORDER BY ratio"
    ).df()
    display(rows.pivot(index='ratio', values='v', columns='index_type'))
plot_for_dataset('fb')


index_type,FLAT_PGM,PGM,SAMPLED_PGM
ratio,,,
1,2.627557,2.957333,68.938314
10,0.418139,0.690873,7.968651
100,0.128248,0.143930,0.775751
1000,0.070089,0.086071,0.123351


index_type,FLAT_PGM,PGM,SAMPLED_PGM
ratio,,,
1,2.627557,2.957333,68.938314
10,0.418139,0.690873,7.968651
100,0.128248,0.143930,0.775751
1000,0.070089,0.086071,0.123351
